In [2]:

import pandas as pd
import numpy as np

import json
import os

import re
import pymongo

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

In [97]:
boston_drinks = pd.read_csv('cocktails_cleaned.csv')
boston_drinks.head()


,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,special ingredients,instructions,glass,glass-size (oz)
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Cherry,Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.,Old-Fashioned Glass,6.0
1,Fort Lauderdale,Cocktail Classics,1.5 oz,Light Rum,.5 oz,Sweet Vermouth,.25 oz,Juice of Orange,.25 oz,Lime Juice,NaN,NaN,NaN,NaN,Orange Slice,Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.,Old-Fashioned Glass,6.0
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Apple Slice,Pour into ice-filled old-fashioned glass. Garnish with an apple slice and top with cinnamon.,Old-Fashioned Glass,6.0
3,Cuban Cocktail No. 1,Cocktail Classics,2 oz,Light Rum,.5 oz,Powdered Sugar,.5 oz,Juice of a Lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6.0
4,Cool Carlos,Cocktail Classics,1.5 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Pineapple Slice, Orange Slice, Cherry","Mix all ingredients except curacao with ice, shake well. Pour into collins glass and top with curacao. Garnish with a pineapple slice, an orange slice, and a cherry.",Collins Glass,14.0


In [98]:
boston_drinks = boston_drinks.fillna('')

In [99]:
cols = list(boston_drinks)
for i, col in enumerate(cols):
    print(i, col)

0 name
1 category
2 measurement-1
3 ingredient-1
4 measurement-2
5 ingredient-2
6 measurement-3
7 ingredient-3
8 measurement-4
9 ingredient-4
10 measurement-5
11 ingredient-5
12 measurement-6
13 ingredient-6
14 special ingredients
15 instructions
16 glass
17 glass-size (oz)


In [100]:
recipes = []
for index, row in boston_drinks.iterrows():
    name = boston_drinks.loc[index, 'name']
    this_recipe = {}
    this_recipe['name'] = name
    this_recipe['category'] = boston_drinks.loc[index, cols[1]]
    this_recipe['ingredients'] = []
    this_recipe['special_ingredients'] = boston_drinks.loc[index, cols[14]]
    this_recipe['instructions'] = boston_drinks.loc[index, cols[15]]
    if (boston_drinks.loc[index, cols[2]] != '') and (boston_drinks.loc[index, cols[3]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[2]]) + ' ' + str(boston_drinks.loc[index, cols[3]]))
    elif (boston_drinks.loc[index, cols[3]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[4]] != '') and (boston_drinks.loc[index, cols[5]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[4]]) + ' ' + str(boston_drinks.loc[index, cols[5]]))
    elif (boston_drinks.loc[index, cols[5]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[6]] != '') and (boston_drinks.loc[index, cols[7]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[6]]) + ' ' + str(boston_drinks.loc[index, cols[7]]))
    elif (boston_drinks.loc[index, cols[7]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[7]]))
    if (boston_drinks.loc[index, cols[8]] != '') and (boston_drinks.loc[index, cols[9]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[8]]) + ' ' + str(boston_drinks.loc[index, cols[9]]))
    elif (boston_drinks.loc[index, cols[9]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[9]]))
    if (boston_drinks.loc[index, cols[10]] != '') and (boston_drinks.loc[index, cols[11]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[10]]) + ' ' + str(boston_drinks.loc[index, cols[11]]))
    elif (boston_drinks.loc[index, cols[11]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[11]]))
    if (boston_drinks.loc[index, cols[12]] != '') and (boston_drinks.loc[index, cols[13]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[12]]) + ' ' + str(boston_drinks.loc[index, cols[13]]))
    elif (boston_drinks.loc[index, cols[13]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[13]]))
    this_recipe['glass_type'] = boston_drinks.loc[index, cols[16]]
    this_recipe['glass_size'] = str(boston_drinks.loc[index, cols[17]])
    recipes.append(this_recipe)

In [101]:
recipes

[{'name': 'Gauguin',
  'category': 'Cocktail Classics',
  'ingredients': ['2 oz  Light Rum',
   '1 oz  Passion Fruit Syrup',
   '1 oz  Lemon Juice',
   '1 oz  Lime Juice'],
  'special_ingredients': 'Cherry',
  'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6.0'},
 {'name': 'Fort Lauderdale',
  'category': 'Cocktail Classics',
  'ingredients': ['1.5 oz  Light Rum',
   '.5 oz  Sweet Vermouth',
   '.25 oz  Juice of Orange',
   '.25 oz  Lime Juice'],
  'special_ingredients': 'Orange Slice',
  'instructions': 'Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6.0'},
 {'name': 'Apple Pie',
  'category': 'Cordials and Liqueurs',
  'ingredients': ['3 oz  Apple schnapps', '1 oz  Cinnamon schnapps'],
  'special_ingredients': 'Apple Sl

In [102]:
for recipe in recipes:
    ing_vals = []
    for ing in recipe['ingredients']:
#         print(re.sub(r'\D', '', ing))
        try:
            print('no error')
            print(recipe['name'])
            print(ing_vals)
            ing_vals.append(float(re.sub(r'[A-Za-z]', '', ing)))
            print(ing_vals)
        except:
            print('============')
            print('error')
            print(recipe['name'])
            print(ing_vals)
            print('============')
    recipe['total_volume'] = str(sum(ing_vals))
    print(sum(ing_vals))

no error
Gauguin
[]
[2.0]
no error
Gauguin
[2.0]
[2.0, 1.0]
no error
Gauguin
[2.0, 1.0]
[2.0, 1.0, 1.0]
no error
Gauguin
[2.0, 1.0, 1.0]
[2.0, 1.0, 1.0, 1.0]
5.0
no error
Fort Lauderdale
[]
[1.5]
no error
Fort Lauderdale
[1.5]
[1.5, 0.5]
no error
Fort Lauderdale
[1.5, 0.5]
[1.5, 0.5, 0.25]
no error
Fort Lauderdale
[1.5, 0.5, 0.25]
[1.5, 0.5, 0.25, 0.25]
2.5
no error
Apple Pie
[]
[3.0]
no error
Apple Pie
[3.0]
[3.0, 1.0]
4.0
no error
Cuban Cocktail No. 1
[]
[2.0]
no error
Cuban Cocktail No. 1
[2.0]
[2.0, 0.5]
no error
Cuban Cocktail No. 1
[2.0, 0.5]
[2.0, 0.5, 0.5]
3.0
no error
Cool Carlos
[]
[1.5]
no error
Cool Carlos
[1.5]
[1.5, 2.0]
no error
Cool Carlos
[1.5, 2.0]
[1.5, 2.0, 2.0]
no error
Cool Carlos
[1.5, 2.0, 2.0]
[1.5, 2.0, 2.0, 1.0]
no error
Cool Carlos
[1.5, 2.0, 2.0, 1.0]
[1.5, 2.0, 2.0, 1.0, 1.0]
7.5
no error
John Collins
[]
[2.0]
no error
John Collins
[2.0]
[2.0, 1.0]
no error
John Collins
[2.0, 1.0]
[2.0, 1.0, 1.0]
no error
John Collins
[2.0, 1.0, 1.0]
[2.0, 1.0, 1.0, 0.5]
4

Button Hook Cocktail
[]
error
Button Hook Cocktail
[]
no error
Button Hook Cocktail
[]
[0.5]
no error
Button Hook Cocktail
[0.5]
[0.5, 0.5]
no error
Button Hook Cocktail
[0.5, 0.5]
[0.5, 0.5, 0.5]
1.5
no error
Bull's Eye
[]
[1.0]
no error
Bull's Eye
[1.0]
[1.0, 2.0]
no error
Bull's Eye
[1.0, 2.0]
error
Bull's Eye
[1.0, 2.0]
3.0
no error
Brantini
[]
[1.5]
no error
Brantini
[1.5]
[1.5, 1.0]
no error
Brantini
[1.5, 1.0]
[1.5, 1.0, 1.0]
3.5
no error
Heather's Kiss
[]
[2.25]
no error
Heather's Kiss
[2.25]
[2.25, 0.5]
no error
Heather's Kiss
[2.25, 0.5]
[2.25, 0.5, 0.25]
no error
Heather's Kiss
[2.25, 0.5, 0.25]
[2.25, 0.5, 0.25, 1.0]
4.0
no error
Brandy Squirt
[]
[1.5]
no error
Brandy Squirt
[1.5]
[1.5, 1.0]
no error
Brandy Squirt
[1.5, 1.0]
[1.5, 1.0, 1.0]
no error
Brandy Squirt
[1.5, 1.0, 1.0]
error
Brandy Squirt
[1.5, 1.0, 1.0]
3.5
no error
Brandy and Soda
[]
[2.0]
no error
Brandy and Soda
[2.0]
error
Brandy and Soda
[2.0]
2.0
no error
Brandy Milk Punch
[]
[1.0]
no error
Brandy Milk Punc

Salty Dog
[1.5]
[1.5, 5.0]
6.5
no error
Dr. Cook
[]
error
Dr. Cook
[]
no error
Dr. Cook
[]
[1.0]
no error
Dr. Cook
[1.0]
[1.0, 1.0]
no error
Dr. Cook
[1.0, 1.0]
[1.0, 1.0, 1.0]
3.0
no error
Dillatini Cocktail
[]
error
Dillatini Cocktail
[]
no error
Dillatini Cocktail
[]
[0.75]
0.75
no error
Chi-Town Flip
[]
[2.0]
no error
Chi-Town Flip
[2.0]
[2.0, 0.75]
no error
Chi-Town Flip
[2.0, 0.75]
[2.0, 0.75, 0.75]
no error
Chi-Town Flip
[2.0, 0.75, 0.75]
error
Chi-Town Flip
[2.0, 0.75, 0.75]
no error
Chi-Town Flip
[2.0, 0.75, 0.75]
[2.0, 0.75, 0.75, 0.25]
no error
Chi-Town Flip
[2.0, 0.75, 0.75, 0.25]
[2.0, 0.75, 0.75, 0.25, 1.0]
4.75
no error
Chef's Pain
[]
[2.0]
no error
Chef's Pain
[2.0]
[2.0, 0.75]
no error
Chef's Pain
[2.0, 0.75]
[2.0, 0.75, 0.5]
no error
Chef's Pain
[2.0, 0.75, 0.5]
error
Chef's Pain
[2.0, 0.75, 0.5]
3.25
no error
Sampan Shipwreck A
[]
[1.5]
no error
Sampan Shipwreck A
[1.5]
[1.5, 0.5]
no error
Sampan Shipwreck A
[1.5, 0.5]
[1.5, 0.5, 0.5]
no error
Sampan Shipwreck A
[1.5

3.5
no error
Cherry Cooler
[]
[2.0]
no error
Cherry Cooler
[2.0]
error
Cherry Cooler
[2.0]
2.0
no error
Polynesian Cocktail
[]
[1.5]
no error
Polynesian Cocktail
[1.5]
[1.5, 0.75]
no error
Polynesian Cocktail
[1.5, 0.75]
[1.5, 0.75, 0.75]
3.0
no error
Chateau Briand's Rum Cow
[]
[1.25]
no error
Chateau Briand's Rum Cow
[1.25]
[1.25, 1.0]
no error
Chateau Briand's Rum Cow
[1.25, 1.0]
[1.25, 1.0, 2.0]
no error
Chateau Briand's Rum Cow
[1.25, 1.0, 2.0]
[1.25, 1.0, 2.0, 0.75]
5.0
no error
Champagne Velvet
[]
[5.0]
no error
Champagne Velvet
[5.0]
[5.0, 5.0]
10.0
no error
Pink Pussycat
[]
[1.5]
no error
Pink Pussycat
[1.5]
[1.5, 2.0]
no error
Pink Pussycat
[1.5, 2.0]
[1.5, 2.0, 0.5]
4.0
no error
Canadian Pineapple
[]
[1.5]
no error
Canadian Pineapple
[1.5]
[1.5, 1.0]
no error
Canadian Pineapple
[1.5, 1.0]
[1.5, 1.0, 1.0]
no error
Canadian Pineapple
[1.5, 1.0, 1.0]
[1.5, 1.0, 1.0, 0.5]
4.0
no error
Calm Voyage
[]
[0.5]
no error
Calm Voyage
[0.5]
[0.5, 0.5]
no error
Calm Voyage
[0.5, 0.5]
[0.5

no error
Whiskey Flip
[1.0, 1.0, 1.0]
error
Whiskey Flip
[1.0, 1.0, 1.0]
3.0
no error
L.A. Sunrise
[]
[1.0]
no error
L.A. Sunrise
[1.0]
[1.0, 1.0]
no error
L.A. Sunrise
[1.0, 1.0]
[1.0, 1.0, 2.0]
no error
L.A. Sunrise
[1.0, 1.0, 2.0]
[1.0, 1.0, 2.0, 2.0]
no error
L.A. Sunrise
[1.0, 1.0, 2.0, 2.0]
[1.0, 1.0, 2.0, 2.0, 0.25]
6.25
no error
White Rose Cocktail
[]
[0.75]
no error
White Rose Cocktail
[0.75]
[0.75, 1.0]
no error
White Rose Cocktail
[0.75, 1.0]
[0.75, 1.0, 1.0]
no error
White Rose Cocktail
[0.75, 1.0, 1.0]
[0.75, 1.0, 1.0, 0.5]
no error
White Rose Cocktail
[0.75, 1.0, 1.0, 0.5]
[0.75, 1.0, 1.0, 0.5, 1.0]
4.25
no error
White Plush
[]
[2.0]
no error
White Plush
[2.0]
[2.0, 1.0]
no error
White Plush
[2.0, 1.0]
[2.0, 1.0, 1.0]
4.0
no error
White Elephant
[]
[1.5]
no error
White Elephant
[1.5]
[1.5, 1.0]
no error
White Elephant
[1.5, 1.0]
[1.5, 1.0, 1.0]
3.5
no error
Whiskey Toddy (cold)
[]
[0.5]
no error
Whiskey Toddy (cold)
[0.5]
[0.5, 2.0]
no error
Whiskey Toddy (cold)
[0.5, 2.0

no error
Egg Sour
[]
[1.0]
no error
Egg Sour
[1.0]
[1.0, 1.0]
no error
Egg Sour
[1.0, 1.0]
error
Egg Sour
[1.0, 1.0]
no error
Egg Sour
[1.0, 1.0]
[1.0, 1.0, 0.25]
2.25
no error
White Lion Cocktail
[]
[1.5]
no error
White Lion Cocktail
[1.5]
[1.5, 1.0]
no error
White Lion Cocktail
[1.5, 1.0]
error
White Lion Cocktail
[1.5, 1.0]
no error
White Lion Cocktail
[1.5, 1.0]
[1.5, 1.0, 2.0]
4.5
no error
White Lily Cocktail
[]
[0.75]
no error
White Lily Cocktail
[0.75]
[0.75, 0.75]
no error
White Lily Cocktail
[0.75, 0.75]
error
White Lily Cocktail
[0.75, 0.75]
1.5
no error
Ice Cream Flip
[]
[1.0]
no error
Ice Cream Flip
[1.0]
[1.0, 1.0]
no error
Ice Cream Flip
[1.0, 1.0]
error
Ice Cream Flip
[1.0, 1.0]
2.0
no error
Outrigger
[]
error
Outrigger
[]
no error
Outrigger
[]
[1.0]
1.0
no error
Opera Cocktail
[]
[1.0]
no error
Opera Cocktail
[1.0]
[1.0, 0.5]
1.5
no error
Virginia Dare
[]
[2.0]
no error
Virginia Dare
[2.0]
[2.0, 2.0]
no error
Virginia Dare
[2.0, 2.0]
error
Virginia Dare
[2.0, 2.0]
4.0
n

error
Cappuccino Cocktail
[0.75]
no error
Cappuccino Cocktail
[0.75]
error
Cappuccino Cocktail
[0.75]
no error
Cappuccino Cocktail
[0.75]
[0.75, 5.0]
5.75
no error
Cape Codder
[]
[1.5]
no error
Cape Codder
[1.5]
[1.5, 5.0]
no error
Cape Codder
[1.5, 5.0]
[1.5, 5.0, 1.0]
no error
Cape Codder
[1.5, 5.0, 1.0]
error
Cape Codder
[1.5, 5.0, 1.0]
no error
Cape Codder
[1.5, 5.0, 1.0]
[1.5, 5.0, 1.0, 1.0]
8.5
no error
Baltimore Bracer Cocktail
[]
error
Baltimore Bracer Cocktail
[]
no error
Baltimore Bracer Cocktail
[]
error
Baltimore Bracer Cocktail
[]
0
no error
Caesar
[]
[1.5]
no error
Caesar
[1.5]
error
Caesar
[1.5]
no error
Caesar
[1.5]
error
Caesar
[1.5]
1.5
no error
Bull Shot
[]
[1.5]
no error
Bull Shot
[1.5]
[1.5, 3.0]
no error
Bull Shot
[1.5, 3.0]
error
Bull Shot
[1.5, 3.0]
no error
Bull Shot
[1.5, 3.0]
[1.5, 3.0, 1.0]
5.5
no error
Bullfrog
[]
[1.5]
no error
Bullfrog
[1.5]
[1.5, 5.0]
6.5
no error
Boston Gold
[]
[1.5]
no error
Boston Gold
[1.5]
[1.5, 0.5]
2.0
no error
Corpse Reviver No. 

[1.0, 0.5, 0.5]
no error
Mariposa
[1.0, 0.5, 0.5]
[1.0, 0.5, 0.5, 0.5]
no error
Mariposa
[1.0, 0.5, 0.5, 0.5]
[1.0, 0.5, 0.5, 0.5, 1.0]
3.5
no error
Mandeville
[]
[1.5]
no error
Mandeville
[1.5]
[1.5, 1.0]
no error
Mandeville
[1.5, 1.0]
[1.5, 1.0, 0.5]
no error
Mandeville
[1.5, 1.0, 0.5]
[1.5, 1.0, 0.5, 1.0]
no error
Mandeville
[1.5, 1.0, 0.5, 1.0]
[1.5, 1.0, 0.5, 1.0, 1.0]
no error
Mandeville
[1.5, 1.0, 0.5, 1.0, 1.0]
error
Mandeville
[1.5, 1.0, 0.5, 1.0, 1.0]
5.0
no error
K.C.B. Cocktail
[]
error
K.C.B. Cocktail
[]
no error
K.C.B. Cocktail
[]
error
K.C.B. Cocktail
[]
no error
K.C.B. Cocktail
[]
[0.25]
no error
K.C.B. Cocktail
[0.25]
[0.25, 0.25]
0.5
no error
Malmaison
[]
error
Malmaison
[]
no error
Malmaison
[]
[1.0]
no error
Malmaison
[1.0]
[1.0, 0.5]
no error
Malmaison
[1.0, 0.5]
[1.0, 0.5, 1.0]
2.5
no error
Mai-Tai (Trader Vic Version)
[]
error
Mai-Tai (Trader Vic Version)
[]
no error
Mai-Tai (Trader Vic Version)
[]
[0.5]
no error
Mai-Tai (Trader Vic Version)
[0.5]
[0.5, 1.0]
no e

Will Rogers
[1.5, 0.5]
[1.5, 0.5, 0.5]
no error
Will Rogers
[1.5, 0.5, 0.5]
[1.5, 0.5, 0.5, 1.0]
3.5
no error
Brandy Vermouth Cocktail
[]
[2.0]
no error
Brandy Vermouth Cocktail
[2.0]
[2.0, 0.5]
no error
Brandy Vermouth Cocktail
[2.0, 0.5]
[2.0, 0.5, 1.0]
3.5
no error
White Spider
[]
[1.0]
no error
White Spider
[1.0]
[1.0, 0.75]
no error
White Spider
[1.0, 0.75]
[1.0, 0.75, 0.5]
no error
White Spider
[1.0, 0.75, 0.5]
[1.0, 0.75, 0.5, 0.25]
2.5
no error
White Lady
[]
[1.5]
no error
White Lady
[1.5]
[1.5, 0.75]
2.25
no error
Western Rose
[]
[1.0]
no error
Western Rose
[1.0]
[1.0, 0.5]
no error
Western Rose
[1.0, 0.5]
error
Western Rose
[1.0, 0.5]
no error
Western Rose
[1.0, 0.5]
[1.0, 0.5, 1.0]
2.5
no error
May Blosson Fizz
[]
[1.0]
no error
May Blosson Fizz
[1.0]
[1.0, 0.5]
no error
May Blosson Fizz
[1.0, 0.5]
[1.0, 0.5, 2.0]
3.5
no error
Wembly Cocktail
[]
[1.5]
no error
Wembly Cocktail
[1.5]
[1.5, 0.75]
no error
Wembly Cocktail
[1.5, 0.75]
[1.5, 0.75, 0.5]
no error
Wembly Cocktail
[1.

In [40]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.cocktail_db

collection = mongoDb.recipe_dump

collection.insert_many(recipes)

In [124]:
collection = mongoDb.glass_svgs
glass_db_response = collection.find({}, {'_id': False})


In [125]:
glasses = []
for glass in glass_db_response:
    glasses.append(glass)


In [126]:
glasses_df = pd.DataFrame(glasses)
glasses_df.head()

,area,glass_type,mask,maskHeight,maskTopMargin,path
0,696.0,cocktail_xl_margarita,"m 43.972082,0.754995 c -0.0081,0.25817 -0.06439,3.2004896 -0.690469,4.62966 -0.724157,1.6530631 -1.930358,3.1629367 -3.394818,4.21763 -1.690209,1.217276 -4.323698,0.765341 -5.858632,2.17337 -1.036281,0.950604 -1.313166,2.495933 -1.802352,3.81435 -0.988876,2.665145 -0.979693,5.686922 -2.26786,8.22099 -0.705615,1.38808 -1.227934,2.26968 -2.906089,3.6574 -1.774973,1.25862 -2.436295,1.05518 -4.809554,1.08607 -2.373258,-0.0337 -2.481197,0.16973 -4.256169,-1.08889 -1.678156,-1.38771 -2.200474,-2.269313 -2.90609,-3.65739 C 13.791881,21.274118 13.801068,18.25234 12.812191,15.587195 12.323005,14.268778 12.04612,12.723449 11.009838,11.772845 9.4749038,10.364817 6.841415,10.816751 5.1512059,9.599475 3.6867458,8.5447817 2.483907,7.0334313 1.7563879,5.381845 1.1285688,3.9565938 1.1109849,1.278665 1.0483987,0.763595 c 12.2431973,0.0655 29.9732023,0.0114 42.9236833,-0.009 z",28.0,1.0,"m 18.233415,27.465585 c -1.678156,-1.38771 -2.200474,-2.269313 -2.90609,-3.65739 -1.288168,-2.534067 -1.278981,-5.555845 -2.267858,-8.22099 -0.489186,-1.318417 -0.766071,-2.863746 -1.802353,-3.81435 C 9.7221797,10.364827 7.0886909,10.816761 5.3984817,9.599485 3.9340216,8.5447918 2.7278527,7.0349042 2.0036637,5.381855 1.3204627,3.8223654 1.3468507,0.88826877 1.2500347,0.72690877 c -0.136137,-0.2269 -0.70336188,-0.26995 -0.79352288,-0.0212 C 0.3161398,1.0930188 0.54700272,4.180746 1.3069577,5.909285 c 0.7263491,1.6521012 1.9303579,3.1629368 3.394818,4.21763 1.6902092,1.217276 4.323698,0.765342 5.8586323,2.17337 1.036282,0.950604 1.313167,2.495933 1.802353,3.81435 0.988877,2.665145 0.972472,5.690605 2.267858,8.221 0.93924,1.834694 2.196101,3.43148 3.874257,4.81919 1.774972,1.25862 2.173361,2.572 2.173361,3.02381 v 20.76082 c 0,0 -0.652167,0.27187 -1.039434,0.75595 -0.322726,0.25818 -0.377976,1.41741 -0.377976,1.41741 L 8.8252247,58.466918 c -0.5809,0.29045 -0.398941,1.20985 -0.398941,1.20985 H 36.552884 c 0,0 0.181959,-0.9194 -0.398941,-1.20985 L 25.718341,55.112815 c 0,0 -0.05525,-1.15923 -0.377976,-1.41741 -0.387267,-0.48408 -1.039434,-0.75595 -1.039434,-0.75595 v -20.76082 c 0,-0.45181 0.398389,-1.76519 2.173361,-3.02381 1.678156,-1.38771 2.935017,-2.984496 3.874257,-4.81919 1.295386,-2.530395 1.278981,-5.555855 2.267858,-8.221 0.489186,-1.318417 0.766071,-2.863746 1.802353,-3.81435 1.534934,-1.408028 4.168423,-0.956094 5.858632,-2.17337 1.46446,-1.0546932 2.668469,-2.5655288 3.394818,-4.21763 0.759955,-1.728539 0.990818,-4.8162662 0.850446,-5.20357623 -0.09016,-0.24875 -0.657386,-0.2057 -0.793523,0.0212 -0.09682,0.16136 -0.07043,3.09545663 -0.753629,4.65494623 -0.724189,1.6530492 -1.930358,3.1629368 -3.394818,4.21763 -1.690209,1.217276 -4.323698,0.765342 -5.858632,2.17337 -1.036282,0.950604 -1.313167,2.495933 -1.802353,3.81435 -0.988877,2.665145 -0.97969,5.686923 -2.267858,8.22099 -0.705616,1.388077 -1.227934,2.26968 -2.90609,3.65739 -1.774972,1.25862 -2.791752,1.08888 -4.256169,1.08888 -1.464417,0 -2.481197,0.16974 -4.256169,-1.08888 z"
1,369.0,champagne_flute,"M 20.289051,33.296931 C 19.290178,31.958842 18.947287,30.927928 18.448495,29.666 17.948006,28.399778 17.611572,27.071254 17.305287,25.744606 16.958789,24.243781 16.72397,22.717532 16.52295,21.190402 16.344847,19.837367 16.224035,18.476064 16.148302,17.113461 16.022373,14.847729 16.029278,12.576249 16.016182,10.307058 15.999277,7.3779968 16.085302,1.52 16.085302,1.52 H 28.989 c 0,0 0.08575,5.857998 0.06879,8.787058 -0.01314,2.26919 -0.0062,4.540671 -0.13212,6.806403 -0.07573,1.362603 -0.196545,2.723906 -0.374648,4.076941 -0.20102,1.52713 -0.435844,3.053378 -0.782337,4.554204 -0.306281,1.326647 -0.642695,2.655175 -1.143182,3.921394 -0.498793,1.261932 -0.841709,2.292842 -1.840582,3.630931 -0.409016,0.547918 -1.172923,1.05518 -2.247936,0.95979 -1.075013,0.09539 -1.83892,-0.411872 -2.247936,-0.95979 z",33.0,1.5,"M 20.229203,33.296931 C 19.23033,31.958842 18.887414,30.927932 18.388621,29.666 17.888134,28.399781 17.55172,27.071

In [72]:
lim_df = glasses_df[['name', 'area']]
lim_df = lim_df.sort_values(by='name')
lim_df

,name,area
4,ROCK,1416.0
14,champagne_extra_fluted,261.0
1,champagne_flute,369.0
6,cocktail_lg,625.0
7,cocktail_md,600.0
2,cocktail_sm_martini,546.0
0,cocktail_xl_margarita,696.0
5,collins,1004.0
12,fluted_lager,774.0
11,highball,1100.0


In [103]:
boston_drinks['glass'].unique()

array(['Old-Fashioned Glass', 'Cocktail Glass', 'Collins Glass',
       'champagne_flute', 'champagne_extra_fluted', 'Highball Glass',
       'irish_coffee', 'ROCK', 'sour', 'poco_grande', 'rocks',
       'hurricane', 'cocktail_sm_martini', 'pousse_cafe', ''],
      dtype=object)

In [104]:
recipes_df = pd.DataFrame(recipes)
recipes_df

,category,glass_size,glass_type,ingredients,instructions,name,special_ingredients,total_volume
0,Cocktail Classics,6.0,Old-Fashioned Glass,"[2 oz Light Rum, 1 oz Passion Fruit Syrup, 1 oz Lemon Juice, 1 oz Lime Juice]",Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.,Gauguin,Cherry,5.0
1,Cocktail Classics,6.0,Old-Fashioned Glass,"[1.5 oz Light Rum, .5 oz Sweet Vermouth, .25 oz Juice of Orange, .25 oz Lime Juice]",Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.,Fort Lauderdale,Orange Slice,2.5
2,Cordials and Liqueurs,6.0,Old-Fashioned Glass,"[3 oz Apple schnapps, 1 oz Cinnamon schnapps]",Pour into ice-filled old-fashioned glass. Garnish with an apple slice and top with cinnamon.,Apple Pie,Apple Slice,4.0
3,Cocktail Classics,6.0,Cocktail Glass,"[2 oz Light Rum, .5 oz Powdered Sugar, .5 oz Juice of a Lime]",Shake with ice and strain into cocktail glass.,Cuban Cocktail No. 1,,3.0
4,Cocktail Classics,14.0,Collins Glass,"[1.5 oz Dark rum, 2 oz Cranberry Juice, 2 oz Pineapple Juice, 1 oz Orange curacao, 1 oz Sour Mix]","Mix all ingredients except curacao with ice, shake well. Pour into collins glass and top with curacao. Garnish with a pineapple slice, an orange slice, and a cherry.",Cool Carlos,"Pineapple Slice, Orange Slice, Cherry",7.5
5,Whiskies,14.0,Collins Glass,"[2 oz Bourbon whiskey, 1 oz lemon juice, 1 oz Soda water, .5 oz Simple Syrup]","Shake first three ingredients with ice and strain into Collins glass. Add ice, fill with soda water, and stir. Garnish with an orange slice, a lemon slice , and a cherry. Serve with straws.",John Collins,"Orange Slice, Lemon Slice, Cherry",4.5
6,Cocktail Classics,6.0,Cocktail Glass,"[1.5 oz cherry-flavored brandy, 1.25 oz Light Rum, 1 oz Light cream]",Shake with ice and strain into cocktail glass.,Cherry Rum,,2.25
7,Cocktail Classics,6.0,Cocktail Glass,"[2 oz Light Rum, 1.5 oz Lime Juice, 1.5 oz Triple Sec, 1.5 oz Maraschino]",Shake with ice and strain into cocktail glass.,Casa Blanca,,6.5
8,Cocktail Classics,6.0,champagne_flute,"[3 Chilled Champagne, .5 oz Creme de banana, .5 oz Light Rum]",Pour rum and banana liqueur into champagne flute. Fill with champagne and stir lightly. Garnish with a banana slice.,Caribbean Champagne,Banana Slice,4.0
9,Cordials and Liqueurs,14.0,Collins Glass,"[1.5 oz Amaretto, .25 oz lemon juice, .25 oz Simple Syrup, 6 Soda water]","Pour amaretto, lemon juice, and syrup into ice-filled Collins glass. Top with soda water and stir. Garnish with a cherry.",Amber Amour,Cherry,8.0


In [116]:
recipes_df.loc[recipes_df['glass_type'] == 'Cocktail Glass', :].groupby(['total_volume', 'glass_type']).count()

,,category,glass_size,ingredients,instructions,name,special_ingredients
total_volume,glass_type,,,,,,
0.00,Cocktail Glass,12,12,12,12,12,12
0.33,Cocktail Glass,1,1,1,1,1,1
0.50,Cocktail Glass,12,12,12,12,12,12
0.75,Cocktail Glass,16,16,16,16,16,16
1.00,Cocktail Glass,21,21,21,21,21,21
1.25,Cocktail Glass,9,9,9,9,9,9
1.50,Cocktail Glass,30,30,30,30,30,30
1.75,Cocktail Glass,4,4,4,4,4,4
1.99,Cocktail Glass,2,2,2,2,2,2


In [112]:
recipes_df['total_volume'] = [float(volume) for volume in recipes_df['total_volume']]
cocktail_glass = recipes_df.loc[recipes_df['glass_type'] == 'Collins Glass', :].sort_values(by='total_volume', ascending=False)
cocktail_glass

,category,glass_size,glass_type,ingredients,instructions,name,special_ingredients,total_volume
160,Cocktail Classics,14.0,Collins Glass,"[5 Sprigs Mint, 1 oz Powdered Sugar, 2 oz, 2.5 oz Bourbon]","In silver mug or collins glass, dissolve powdered sugar with water. Fill with finely shaved ice and add bourbon. Stir until glass is heavily frosted, adding more ice if necessary. (Do not hold glass with hand while stirring.) Garnish with mint so that the tops are about two inches above the rim of glass. Use short straws so that it will be necessary to bury nose in mint, which is intended for scent rather than taste.",Mint Julep (Southern Style),Mint,10.50
314,Cocktail Classics,14.0,Collins Glass,"[2 oz Vodka, 1 oz Egg Yolk, 6 oz, 1 oz Sugar]",Combine all ingredients with a half-cup of crushed ice in an electric blender. Blend at low speed and pour into collins glass.,Creamy Screwdriver,,10.00
695,Rum - Daiquiris,14.0,Collins Glass,"[3 oz Lime wedges, 7 mint leaves, .75 oz 2 oz, Soda water]","Muddle the limes, mint, and syrup in a Collins glass. Add the rum with ice and stir well. Top with soda water. Garnish with mint.",Mojito,Mint,10.00
306,Cocktail Classics,14.0,Collins Glass,"[1 oz Juice of a Lemon, 2 oz Powdered Sugar, 1 oz, 6 oz Water]","Shake with ice and strain into collins glass. Garnish with an orange slice, a lemon slice, and a cherry.",Lemonade (Golden),"Orange Slice, Lemon Slice, Cherry",10.00
739,Tequila,14.0,Collins Glass,"[3 oz Cucumber, 3 oz sage leaves, .5 oz 1.5 oz, .75 oz Simple Syrup, 3 oz]","Muddle cucumber, sage, and lime juice in mixing glass. Add tequila and syrup with ice and shake. Double-strain into ice-filled Collins glass. Top with ginger ale. Garnish with a strawberry.",Waiting On Summer,Strawberry,9.75
179,Cocktail Classics,14.0,Collins Glass,"[3 oz Amaretto di Saronno, 3 oz Coconut Milk, 3 oz]",Put in an electric blender with 2 cups of crushed ice and blend at high speed for a short time. Pour into a collins glass and serve with a straw.,Saronno Colada (Nutty Colada),,9.00
445,Cocktail Classics,14.0,Collins Glass,"[1 oz Juice of a Lime, 1 oz Powdered Sugar, 2 oz, 2 oz Gin, 2 oz Bitters, 1 oz Grenadine]","Combine first four ingredients in ice-filled Collins glass. With bar spoon between your palms, move hands back and forth and up and down to quickly rotate and lift spoon, until glass is frosted. Top with soda water and add one tablespoon of grenadine. If desired, rum, brandy, or whiskey may be substituted for the gin.",Red Swizzle,,9.00
9,Cordials and Liqueurs,14.0,Collins Glass,"[1.5 oz Amaretto, .25 oz lemon juice, .25 oz Simple Syrup, 6 Soda water]","Pour amaretto, lemon juice, and syrup into ice-filled Collins glass. Top with soda water and stir. Garnish with a cherry.",Amber Amour,Cherry,8.00
730,Vodka,14.0,Collins Glass,"[3 oz basil leaves, 5 oz White grapes, 1.5 oz .75 oz, 1 oz each Simple Syrup, angostura bitters]",Muddle basil and 5 grapes in Collins glass. Add next four ingredients and stir. Add ice and top with ginger ale. Garnish with basil sprig and a white grape.,Basil 8,"Basil, Grape",8.00
425,Vodka,14.0,Collins Glass,"[4 oz basil leaves, 1.5 oz Vodka, .5 oz, .5 oz Lime Juice, .5 oz Simple Syrup, 1 oz Passion Fruit Juice]",Muddle 4 basil leaves in mixing glass. Add remaining ingredients. Shake with ice. Double-strain through wire sieve into crushed ice-filled Collins glass. Garnish with a basil leaf.,Maracuya Mosquito,Basil,8.00


In [123]:
recipes_df['glass_type'].value_counts()

cocktail_md               288
cocktail_sm_martini       108
cocktail_lg               104
Old-Fashioned Glass       98 
collins                   98 
fluted_lager              50 
pilsner                   43 
                          38 
sour                      23 
champagne_flute           20 
highball                  19 
old_fashioned             17 
cocktail_xl_margarita     17 
poco_grande               16 
hurricane                 11 
ROCK                      9  
rocks                     7  
champagne_extra_fluted    6  
irish_coffee              3  
pousse_cafe               1  
Name: glass_type, dtype: int64

In [122]:
for index, row in recipes_df.iterrows():
    if recipes_df.loc[index, 'glass_type'] == 'Highball Glass' and recipes_df.loc[index, 'total_volume'] < 3:
        recipes_df.loc[index, 'glass_type'] = 'pilsner'
    if recipes_df.loc[index, 'glass_type'] == 'Highball Glass' and recipes_df.loc[index, 'total_volume'] < 6:
        recipes_df.loc[index, 'glass_type'] = 'fluted_lager'
    if recipes_df.loc[index, 'glass_type'] == 'Highball Glass' and recipes_df.loc[index, 'total_volume'] >= 6:
        recipes_df.loc[index, 'glass_type'] = 'collins'
    if recipes_df.loc[index, 'glass_type'] == 'Collins Glass' and recipes_df.loc[index, 'total_volume'] < 2:
        recipes_df.loc[index, 'glass_type'] = 'fluted_lager'
    if recipes_df.loc[index, 'glass_type'] == 'Collins Glass' and recipes_df.loc[index, 'total_volume'] < 6:
        recipes_df.loc[index, 'glass_type'] = 'collins'
    if recipes_df.loc[index, 'glass_type'] == 'Collins Glass' and recipes_df.loc[index, 'total_volume'] >= 6:
        recipes_df.loc[index, 'glass_type'] = 'highball'
    if recipes_df.loc[index, 'glass_type'] == 'Cocktail Glass' and recipes_df.loc[index, 'total_volume'] < 2:
        recipes_df.loc[index, 'glass_type'] = 'cocktail_sm_martini'
    if recipes_df.loc[index, 'glass_type'] == 'Cocktail Glass' and recipes_df.loc[index, 'total_volume'] < 4:
        recipes_df.loc[index, 'glass_type'] = 'cocktail_md'
    if recipes_df.loc[index, 'glass_type'] == 'Cocktail Glass' and recipes_df.loc[index, 'total_volume'] < 6:
        recipes_df.loc[index, 'glass_type'] = 'cocktail_lg'
    if recipes_df.loc[index, 'glass_type'] == 'Cocktail Glass' and recipes_df.loc[index, 'total_volume'] < 8:
        recipes_df.loc[index, 'glass_type'] = 'cocktail_xl_margarita'
    if recipes_df.loc[index, 'glass_type'] == 'Cocktail Glass' and recipes_df.loc[index, 'total_volume'] >= 8:
        recipes_df.loc[index, 'glass_type'] = 'rocks'
    if recipes_df.loc[index, 'glass_type'] == 'Old-Fashioned Glass' and recipes_df.loc[index, 'total_volume'] < 2:
        recipes_df.loc[index, 'glass_type'] = 'old_fashioned'
    

In [128]:
merged = pd.merge(glasses_df, recipes_df, on='glass_type', how='right')
merged.head()

,area,glass_type,mask,maskHeight,maskTopMargin,path,category,glass_size,ingredients,instructions,name,special_ingredients,total_volume
0,696.0,cocktail_xl_margarita,"m 43.972082,0.754995 c -0.0081,0.25817 -0.06439,3.2004896 -0.690469,4.62966 -0.724157,1.6530631 -1.930358,3.1629367 -3.394818,4.21763 -1.690209,1.217276 -4.323698,0.765341 -5.858632,2.17337 -1.036281,0.950604 -1.313166,2.495933 -1.802352,3.81435 -0.988876,2.665145 -0.979693,5.686922 -2.26786,8.22099 -0.705615,1.38808 -1.227934,2.26968 -2.906089,3.6574 -1.774973,1.25862 -2.436295,1.05518 -4.809554,1.08607 -2.373258,-0.0337 -2.481197,0.16973 -4.256169,-1.08889 -1.678156,-1.38771 -2.200474,-2.269313 -2.90609,-3.65739 C 13.791881,21.274118 13.801068,18.25234 12.812191,15.587195 12.323005,14.268778 12.04612,12.723449 11.009838,11.772845 9.4749038,10.364817 6.841415,10.816751 5.1512059,9.599475 3.6867458,8.5447817 2.483907,7.0334313 1.7563879,5.381845 1.1285688,3.9565938 1.1109849,1.278665 1.0483987,0.763595 c 12.2431973,0.0655 29.9732023,0.0114 42.9236833,-0.009 z",28.0,1.0,"m 18.233415,27.465585 c -1.678156,-1.38771 -2.200474,-2.269313 -2.90609,-3.65739 -1.288168,-2.534067 -1.278981,-5.555845 -2.267858,-8.22099 -0.489186,-1.318417 -0.766071,-2.863746 -1.802353,-3.81435 C 9.7221797,10.364827 7.0886909,10.816761 5.3984817,9.599485 3.9340216,8.5447918 2.7278527,7.0349042 2.0036637,5.381855 1.3204627,3.8223654 1.3468507,0.88826877 1.2500347,0.72690877 c -0.136137,-0.2269 -0.70336188,-0.26995 -0.79352288,-0.0212 C 0.3161398,1.0930188 0.54700272,4.180746 1.3069577,5.909285 c 0.7263491,1.6521012 1.9303579,3.1629368 3.394818,4.21763 1.6902092,1.217276 4.323698,0.765342 5.8586323,2.17337 1.036282,0.950604 1.313167,2.495933 1.802353,3.81435 0.988877,2.665145 0.972472,5.690605 2.267858,8.221 0.93924,1.834694 2.196101,3.43148 3.874257,4.81919 1.774972,1.25862 2.173361,2.572 2.173361,3.02381 v 20.76082 c 0,0 -0.652167,0.27187 -1.039434,0.75595 -0.322726,0.25818 -0.377976,1.41741 -0.377976,1.41741 L 8.8252247,58.466918 c -0.5809,0.29045 -0.398941,1.20985 -0.398941,1.20985 H 36.552884 c 0,0 0.181959,-0.9194 -0.398941,-1.20985 L 25.718341,55.112815 c 0,0 -0.05525,-1.15923 -0.377976,-1.41741 -0.387267,-0.48408 -1.039434,-0.75595 -1.039434,-0.75595 v -20.76082 c 0,-0.45181 0.398389,-1.76519 2.173361,-3.02381 1.678156,-1.38771 2.935017,-2.984496 3.874257,-4.81919 1.295386,-2.530395 1.278981,-5.555855 2.267858,-8.221 0.489186,-1.318417 0.766071,-2.863746 1.802353,-3.81435 1.534934,-1.408028 4.168423,-0.956094 5.858632,-2.17337 1.46446,-1.0546932 2.668469,-2.5655288 3.394818,-4.21763 0.759955,-1.728539 0.990818,-4.8162662 0.850446,-5.20357623 -0.09016,-0.24875 -0.657386,-0.2057 -0.793523,0.0212 -0.09682,0.16136 -0.07043,3.09545663 -0.753629,4.65494623 -0.724189,1.6530492 -1.930358,3.1629368 -3.394818,4.21763 -1.690209,1.217276 -4.323698,0.765342 -5.858632,2.17337 -1.036282,0.950604 -1.313167,2.495933 -1.802353,3.81435 -0.988877,2.665145 -0.97969,5.686923 -2.267858,8.22099 -0.705616,1.388077 -1.227934,2.26968 -2.90609,3.65739 -1.774972,1.25862 -2.791752,1.08888 -4.256169,1.08888 -1.464417,0 -2.481197,0.16974 -4.256169,-1.08888 z",Cocktail Classics,6.0,"[2 oz Light Rum, 1.5 oz Lime Juice, 1.5 oz Triple Sec, 1.5 oz Maraschino]",Shake with ice and strain into cocktail glass.,Casa Blanca,,6.50
1,696.0,cocktail_xl_margarita,"m 43.972082,0.754995 c -0.0081,0.25817 -0.06439,3.2004896 -0.690469,4.62966 -0.724157,1.6530631 -1.930358,3.1629367 -3.394818,4.21763 -1.690209,1.217276 -4.323698,0.765341 -5.858632,2.17337 -1.036281,0.950604 -1.313166,2.495933 -1.802352,3.81435 -0.988876,2.665145 -0.979693,5.686922 -2.26786,8.22099 -0.705615,1.38808 -1.227934,2.26968 -2.906089,3.6574 -1.774973,1.25862 -2.436295,1.05518 -4.809554,1.08607 -2.373258,-0.0337 -2.481197,0.16973 -4.256169,-1.08889 -1.678156,-1.38771 -2.200474,-2.269313 -2.90609,-3.65739 C 13.791881,21.274118 13.801068,18.25234 12.812191,15.587195 12.323005,14.268778 12.04612,12.723449 11.009838,11.772845 9.4749038,10.364817 6.841415,10.816751 5.1

In [136]:
merged = merged.fillna('')
# recipes_df['total_volume'] = [str(volume) for volume in recipes_df['total_volume']]

In [137]:
merged_dict = merged.to_dict(orient='records')


In [138]:
collection = mongoDb.recipe_db
collection.insert_many(merged_dict)